In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

login("HF_TOKEN")

In [ ]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset

# Load dataset
ds = load_dataset("Mahadih534/brain-tumor-dataset")

Resolving data files:   0%|          | 0/253 [00:00<?, ?it/s]

In [ ]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import time
from groq import Groq
import os
import base64
from io import BytesIO
from PIL import Image

# Initialize Groq client
client = Groq(api_key='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')

# Load dataset
print("Loading dataset...")
ds = load_dataset("Mahadih534/brain-tumor-dataset")

# Examine the dataset structure
print("Examining dataset structure...")
example = ds["train"][0]
print("Dataset columns:", list(example.keys()))

# Print a few examples to understand the label structure
print("\nExamining labels in the first few examples:")
for i in range(min(5, len(ds["train"]))):
    example = ds["train"][i]
    print(f"Example {i+1}:")
    if "label" in example:
        print(f"  label: {example['label']}")
    elif "class" in example:
        print(f"  class: {example['class']}")
    elif "annotations" in example and example["annotations"]:
        print(f"  annotations: {example['annotations']}")
    else:
        print("  No clear label field found")

def get_tumor_status(example):
    """Extract tumor status directly from label column"""
    # Check label field first
    if "label" in example:
        value = str(example["label"]).lower()
        if value in ["yes", "1", "true", "tumor"]:
            return "yes"
        elif value in ["no", "0", "false", "normal"]:
            return "no"

    # If not found in label, check class field
    if "class" in example:
        value = str(example["class"]).lower()
        if value in ["yes", "1", "true", "tumor"]:
            return "yes"
        elif value in ["no", "0", "false", "normal"]:
            return "no"

    # If still not found, check annotations
    if "annotations" in example and example["annotations"]:
        for ann in example["annotations"]:
            if "label" in ann:
                value = str(ann["label"]).lower()
                if value in ["yes", "1", "true", "tumor"]:
                    return "yes"
                elif value in ["no", "0", "false", "normal"]:
                    return "no"

    # Return None if we can't determine
    return None

def image_to_base64(pil_image):
    """Convert PIL image to base64 string"""
    buffered = BytesIO()
    pil_image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def generate_vision_caption(image):
    """Generate caption using llama-3.2-11b-vision-preview model by scanning the image"""
    # Convert PIL image to base64
    base64_image = image_to_base64(image)

    try:
        # Using the llama-3.2-11b-vision-preview model
        completion = client.chat.completions.create(
            model="llama-3.2-11b-vision-preview",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "You are a medical imaging expert. Analyze this brain scan image in detail. Describe any visible structures, abnormalities, or features that would be relevant for medical professionals. Be specific about any notable areas or patterns. Keep your analysis concise (2-3 sentences) but technically precise."
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            temperature=0.5,
            max_completion_tokens=200,
            top_p=1,
            stream=False,
            stop=None,
        )

        # Extract the generated caption
        caption = completion.choices[0].message.content.strip()
        return caption

    except Exception as e:
        print(f"Error generating caption with vision model: {e}")
        raise e

def process_example(example):
    """Process a single example with vision model caption"""
    # Determine tumor status first
    tumor_status = get_tumor_status(example)
    if tumor_status not in ["yes", "no"]:
        print(f"Warning: Unable to determine tumor status for example")
        return None

    status_text = "YES (tumor present)" if tumor_status == "yes" else "NO (no tumor)"
    print(f"Tumor status determined: {status_text}")

    try:
        # Generate caption using vision model based on actual image
        image = example["image"]
        vision_caption = generate_vision_caption(image)

        # Create final text with tumor status followed by vision-generated caption
        tumor_prefix = "Brain tumor present. " if tumor_status == "yes" else "No brain tumor detected. "
        example["text"] = tumor_prefix + vision_caption

        print(f"Caption generated: {example['text'][:100]}...")
        return example

    except Exception as e:
        print(f"Error processing example: {e}")
        # We don't want default captions, so return None on error
        return None

# Process a small batch first as a test
print("\nProcessing test batch...")
test_ds = ds["train"].select(range(min(3, len(ds["train"]))))
test_processed = []

for i in range(len(test_ds)):
    processed = process_example(test_ds[i])
    if processed:
        test_processed.append(processed)

# Display the generated captions for the test batch
print("\nGenerated captions for test batch:")
for i, example in enumerate(test_processed):
    tumor_status = get_tumor_status(example)
    status_text = "YES (tumor present)" if tumor_status == "yes" else "NO (no tumor)"
    print(f"Example {i+1} - Status: {status_text}")
    print(f"Caption: {example['text']}")
    print()

# Process the full dataset
print("\nProcessing full dataset...")

# Process with rate limiting to avoid API throttling
def process_full_dataset(dataset, batch_size=3, delay=5):
    """Process the full dataset with rate limiting"""
    from datasets import Dataset

    all_examples = []

    for i in range(0, len(dataset)):
        if i % batch_size == 0:
            print(f"Processing examples {i+1}-{min(i+batch_size, len(dataset))} of {len(dataset)}")

        # Process one example at a time
        try:
            example = dataset[i]
            processed = process_example(example)

            # Only keep successfully processed examples
            if processed:
                all_examples.append(processed)

        except Exception as e:
            print(f"Error processing example {i}: {e}")
            # Skip this example if there's an error - no default captions

        # Add delay between API calls to avoid rate limits
        if (i + 1) % batch_size == 0 and i < len(dataset) - 1:
            time.sleep(delay)

    if not all_examples:
        print("No examples were successfully processed!")
        return None

    # Create a new dataset from the processed examples
    fields = list(all_examples[0].keys())
    result_dict = {field: [ex[field] for ex in all_examples] for field in fields}
    return Dataset.from_dict(result_dict)

# Process each split
processed_ds = {}
for split in ds.keys():
    print(f"Processing {split} split...")
    processed_split = process_full_dataset(ds[split])
    if processed_split:
        processed_ds[split] = processed_split

# To push to hub, you need to be logged in
from huggingface_hub import login
# login()  # Uncomment and run this once to authenticate

# Push dataset to the Hub if any data was successfully processed
if processed_ds and "train" in processed_ds:
    processed_ds["train"].push_to_hub("adityadeshpande03/brain-tumor-captions-llama-vision")
    print("Dataset successfully uploaded to Hugging Face Hub!")
else:
    print("No data to upload. Please check errors above.")

Loading dataset...


Resolving data files:   0%|          | 0/253 [00:00<?, ?it/s]

Using the latest cached version of the dataset since Mahadih534/brain-tumor-dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /root/.cache/huggingface/datasets/Mahadih534___brain-tumor-dataset/default/0.0.0/f5fc43d3c09f062dc67b65796e4623081e2351d9 (last modified on Mon Mar  3 07:14:51 2025).


Examining dataset structure...
Dataset columns: ['image', 'label']

Examining labels in the first few examples:
Example 1:
  label: 0
Example 2:
  label: 0
Example 3:
  label: 0
Example 4:
  label: 0
Example 5:
  label: 0

Processing test batch...
Tumor status determined: NO (no tumor)
Caption generated: No brain tumor detected. The brain scan image exhibits a subtle hyperintensity in the left occipital...
Tumor status determined: NO (no tumor)
Caption generated: No brain tumor detected. The brain scan image reveals a normal brain structure with no visible abnor...
Tumor status determined: NO (no tumor)
Caption generated: No brain tumor detected. The brain scan image reveals a normal brain structure with no visible abnor...

Generated captions for test batch:
Example 1 - Status: NO (no tumor)
Caption: No brain tumor detected. The brain scan image exhibits a subtle hyperintensity in the left occipital lobe, characterized by a slightly elevated signal intensity compared to the surroundin

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/253 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Dataset successfully uploaded to Hugging Face Hub!
